# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [4]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

In [5]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [6]:
# FOR TESTING : only select first 20000 samples
X, y = X[:20000], y[:20000]

In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
#tokenized_documents = tokenizer(X)

Number of tokens:  0
Number of sentences:  0


In [5]:
#X, vect = vectorizer(tokenized_documents)

/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
batch_size = 10000

print(f"Processing data of length {len(X)} in batches of size {batch_size}")
X_vect_batches = []
for start in range(0, len(X), batch_size):
    end = start + batch_size
    print(f"Processing batch from index {start} to {end}")
    batch_X = X[start:end].copy()
    tokenized_documents = tokenizer(batch_X)
    X_vect, _ = vectorizer(tokenized_documents)
    # Store the processed batch
    X_vect_batches.append(X_vect)

X = np.vstack(X_vect_batches)

Processing data of length 20000 in batches of size 10000
Processing batch from index 0 to 10000
Number of tokens:  0
Number of sentences:  0
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Processing batch from index 10000 to 20000
Number of tokens:  0
Number of sentences:  0


In [6]:
# # most frequent words
# print("Top 10 most frequent words in the dataset")
# print(vect.get_feature_names_out()[:10])

# # least frequent words
# print("Top 10 least frequent words in the dataset")
# print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0003' '000kwh' '002' '008' '01' '0100' '0174']
Top 10 least frequent words in the dataset
['zuke' 'zukes' 'zupas' 'zuppa' 'zwieback' 'º' '¼' '½' 'çaykur' 'ît']


In [11]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(16000, 300) (4000, 300) (16000,) (4000,)
[0.31688336 0.58280444 0.4434279  0.68124328 0.70219245 0.40117317
 0.70125687 0.79297498 0.68009419 0.60324099 0.53017887 0.73837851
 0.71468757 0.69045167 0.32775743 0.46927178 0.56479923 0.39001284
 0.59799255 0.44477883 0.42805825 0.69587392 0.51600367 0.5881948
 0.6771487  0.49759001 0.45198786 0.32847894 0.42298602 0.4198587
 0.43506469 0.56132976 0.61844243 0.41619936 0.43391976 0.41204952
 0.31438139 0.41487532 0.37426337 0.60059109 0.59009577 0.48542257
 0.54360355 0.61684623 0.64873781 0.43900139 0.4007275  0.44470909
 0.64421864 0.72376676 0.45985864 0.52836262 0.66623578 0.55054861
 0.66258592 0.29043657 0.57203204 0.22234516 0.23995245 0.468081
 0.35787725 0.60270948 0.21958953 0.41913876 0.61877975 0.52750096
 0.43873732 0.57690618 0.48082081 0.20810526 0.60044096 0.58730211
 0.56438266 0.57753957 0.42443589 0.42803897 0.1928314  0.45568396
 0.41060369 0.51829385 0.60781687 0.50827845 0.34413105 0.76512656
 0.53647165 0.43040991 0

# Feed Forward Neural Network
### Model starts here

In [15]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU
from sklearn import metrics
import datetime
import os

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.00001)
]

In [18]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

def batch_generator(X, y, batch_size, num_classes):
    num_samples = X.shape[0]
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end]
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield X_batch, y_batch

# Setup the model
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(np.max(y_train) + 1, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

# Define the generator
train_generator = batch_generator(X_train, y_train, 32, np.max(y_train) + 1)

2024-05-16 23:16:48.919883: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-16 23:16:49.304789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-16 23:16:49.305048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [20]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=int(np.ceil(X_train.shape[0] / 32)),
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100


2024-05-16 23:17:10.230932: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


500/500 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.6164 - loss: 1.3717 - learning_rate: 1.0000e-04
Epoch 2/100
  5/500 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.6922 - loss: 1.0502

/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


121/500 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.6349 - loss: 1.1539

KeyboardInterrupt: 

In [23]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1)

# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print("Confusion Matrix:")
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

print("\nClassification Report:")
print(metrics.classification_report(y_test, predictions))

125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Confusion Matrix:
[[   0    0    0    0  353]
 [   0    0    0    0  246]
 [   0    0    0    0  342]
 [   0    0    0    0  567]
 [   0    0    0    0 2492]]

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       353
           2       0.00      0.00      0.00       246
           3       0.00      0.00      0.00       342
           4       0.00      0.00      0.00       567
           5       0.62      1.00      0.77      2492

    accuracy                           0.62      4000
   macro avg       0.12      0.20      0.15      4000
weighted avg       0.39      0.62      0.48      4000



/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [24]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config3_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')